In [1]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt

In [14]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
kSZ = np.load('./MAPS/kSZ_NS_1024_R_2968_P_2048_DV_192_Sd_844949.npy')
overdensity = np.load('./MAPS/overdensity_NS_1024_R_2968_P_2048_DV_192_Sd_844949_SB_7.npy')
velocityField = np.load('./MAPS/velocityField_NS_1024_R_2968_P_2048_DV_192_Sd_844949_SB_7.npy')

In [3]:
nside=1024

In [18]:
hp.mollview(kSZ,max=1*10**-5,min=-1*10**-5)

In [19]:
hp.zoomtool.mollzoom(kSZ,max=1*10**-5,min=-1*10**-5)

In [82]:
hp.mollview(overdensity,min=-1,max=2)

In [83]:
hp.zoomtool.mollzoom(overdensity)

In [67]:
hp.mollview(velocityField)

In [68]:
hp.zoomtool.mollzoom(velocityField,title='Truth')

In [79]:
import camb
#import pywigxjpf as pywig
#from joblib import Parallel, delayed
import sys, argparse, multiprocessing
#from common import *
from scipy.signal import savgol_filter


# In[9]:


#Make Fake CMB
h=0.69
pars = camb.CAMBparams()
pars.set_cosmology(H0=100.0*h, ombh2=0.048*h**2, omch2=0.262*h**2, mnu=0.06, omk=0)
pars.InitPower.set_params(As=2e-9, ns=0.96, r=0)
pars.set_for_lmax(6144, lens_potential_accuracy=0)
results = camb.get_results(pars)
powers =results.get_cmb_power_spectra(pars, CMB_unit='K')
l=np.arange(0,len(powers['total'][:,0]))
cambFactor = l*(l+1)/(2*np.pi)
CMB_camb = powers['total'][:,0]/cambFactor
CMB_camb[0]=0.0

#rho map is just the overdensity
def quadEst(ksz_map, rho_map):

    ksz_PS = hp.anafast(ksz_map)

    CMB_map = hp.sphtfunc.synfast(CMB_camb,nside=nside);

    Obs_T_map = ksz_map + CMB_map

    ClTT = hp.anafast(Obs_T_map)
    ClTT_filtered = np.concatenate(([1.0], savgol_filter(ClTT[1:], 51, 3)))

    dTlm = hp.map2alm(Obs_T_map)
    dlm = hp.map2alm(rho_map)

    dTlm_resc = hp.almxfl(dTlm, 1.0/ClTT)
    dT_resc = hp.alm2map(dTlm_resc, nside)
    dlm = -1.0*dlm # dlm_resc = hp.almxfl(dlm, 1.0) #Cltd/Cldd
    d_resc = hp.alm2map(dlm, nside)

    unnorm_veff_reconstlm = hp.map2alm(dT_resc*d_resc)
    unnorm_veff_reconst_ps = hp.alm2cl(unnorm_veff_reconstlm)
    unnorm_veff_reconst = hp.alm2map(unnorm_veff_reconstlm, nside)

    return unnorm_veff_reconst


/tmp/ipykernel_1416/1925348625.py:22: RuntimeWarning: invalid value encountered in true_divide
  CMB_camb = powers['total'][:,0]/cambFactor


In [80]:
reconstruction = quadEst(kSZ,overdensity)

In [71]:
smoothRecon = hp.smoothing(reconstruction, .05)

In [51]:
plt.hist(smoothRecon,bins=np.linspace(-.25*10**10,.25*10**10));

In [59]:
hp.mollview(smoothRecon,min=-.25*10**10,max=.25*10**10,title='Reconstruction')

In [72]:
hp.zoomtool.mollzoom(smoothRecon,title='Reconstruction')

In [81]:
hp.zoomtool.mollzoom(reconstruction,title='Reconstruction')

In [47]:
hp.mollview(reconstruction,min=-.25*10**10,max=.25*10**10,title='Reconstruction')

In [27]:
def Correlation(map1, map2):
    return hp.anafast(map1,map2)/(np.sqrt(hp.anafast(map1)*hp.anafast(map2))+np.finfo(np.float32).eps)

In [57]:
velCorr = Correlation(smoothRecon,velocityField)

In [58]:
plt.semilogx(velCorr)